<a href="https://colab.research.google.com/github/eldend/kh/blob/main/5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [19]:
#훈련세트의 20%를 뗴서 검증세트를 만듦(훈련세트를 한 번 더 나누는 것을 검증세트라고 )
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [10]:
print(sub_input.shape, val_input.shape, test_input.shape)

(4157, 3) (1040, 3) (1300, 3)


In [8]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [11]:
#교차검증: 검증세트를 여러 개 떼어내어 평가를 반복
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
#cross_validate의 경우 test_score의 값만 반환한다.
print(scores)

{'fit_time': array([0.00899911, 0.00723314, 0.00857902, 0.01034689, 0.00924635]), 'score_time': array([0.00119758, 0.00108266, 0.00152588, 0.00149393, 0.00111651]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [18]:
import numpy as np

print(np.mean(scores['test_score']))
#검증 폴드의 점수(혼돈x) -> 테스트 점수가 아님

0.855300214703487


In [17]:
from sklearn.model_selection import StratifiedKFold
#분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratofiedKFold를 사용
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [20]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
#n-splits의 경우 매개변수는 몇(k) 폴드 교차 검증을 할지 정함
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))
#검증세트의 점수가 살짝 높아짐

0.8574181117533719


In [21]:
#하이퍼피라미터 튜닝: 라이브러리가 제공하는 기본값을 그대로 사용하여 훈련 그 다음 검증 세트이 점수나 교차 검증을 통해서 매개변수를 조금씩 바꾸며 모델을 훈련하고 교차 검증을 수행
#하이퍼피라미터: 사용자가 지정해야만 하는 피라미터
from sklearn.model_selection import GridSearchCV
#GridSearchCV 클래스의 경우 하이퍼파라미터 탐새과 교차 검증을 한번에 수행
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
#0.0001씩 증가하는 5개의 값으로 시도
#mim_impurity_decrease의 최적값 찾기

In [24]:
#그리드 서치 객체 생성
#cv매개변수의 기본값은 5
#5-폴드 교차 검증을 수행, 즉 5*5=25개의 모델을 훈련
#n_jobs: 병렬 실행에 사용할 CPU코어 수를 지정하는 매개변수
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [25]:
#그리드 서치 객체 훈련
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [26]:
#그리드 서치의 경우 25개의 모델 중 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련세트에서 자동으로 다시 모델을 훈련
#이 모델은 gs객체의 best_estimator_ 속성에 저장 되어 있음
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [27]:
#그리드 서치로 찾은 최적의 매개변수를 찾는 함수
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [28]:
#각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_속성의 'mean_test_score'키에 저장되어 있음
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [29]:
#첫번째 값의 점수가 가장 큼
#argmax()라는 함수를 이용하여 가장 큰 값의 인덱스를 추출 그 다음 인덱스를 이용하여 params 키에 저장된 매개변수를 출력
#이 값이 최상 검증 점수를 만든 매개변수의 조합이다.
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [30]:
#넘파이 arange()함수는 첫 번째 매개변수 값에서 시작하여 두 번째 매개변수에 도달할 때까지 세 번째 매개변수를 계속 더한 배열을 만듦
#코드에서 0.0001에서 시작하여 0.001이 될 때까지 0.0001을 계속 더한 배열을 만듦
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          #5~20까지 1씩 증가하는 15개의 값을 만듦
          'min_samples_split': range(2, 100, 10)
          #2~100까지 10씩 증가하는 10개의 값을 만듦
          }
#따라서 9*15*10 =1350회의 검증 횟수를 가진다.
#기본5-폴드 교차검증을 수행하므로 만들어지는 모델 수는 1350*5=6750개의 모델이 만들어짐

In [31]:
#n_jons=-1로 설정하고(CPU코어 개수 -1개) 서치 진행
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [34]:
#최상의 매개변수 조합표 확인
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [35]:
#최상의 교차 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


랜덤 서치: 메개변수의 값이 수치일 떄 값의 범위나 간격을 미리 정하기 어려울 경우 사용


In [36]:
#scipy의 stats 서브 패키지에 있는 uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑음
from scipy.stats import uniform, randint

In [37]:
#0!10 샤이의 범위를 갑는 randint 객체를 만들고 10게의 숫자를 샘플링
rgen = randint(0, 10)
rgen.rvs(10)

array([3, 6, 7, 2, 3, 2, 9, 3, 5, 9])

In [41]:
#1000개를 샘플링 해서 고르게 샘플링 됐는지 확인
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 95,  94, 106, 100, 111,  95,  97, 101, 103,  98]))

In [40]:
#unifom클래스의 사용법도 동일
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.42137607, 0.97636903, 0.39228569, 0.18203243, 0.92806952,
       0.59872351, 0.3015701 , 0.42968168, 0.97080808, 0.21445413])

In [42]:
#탐색할 매개변수의 딕셔너리 생성
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [62]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
#샘플링 횟수는 n_iter에 지정(100번 반복)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e1a1153f5b0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7e1a1153db70>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e1a1153dc90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7e1a1153f3a0>},
                   random_state=42)

In [46]:
#최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [57]:
#최상의 검증 점수 확인
print(np.max(gs.cv_results_['mean_test_score']))

0.8458726956392981


In [63]:
#최적 모델은 best_estimator_속성에 저장되어 있기 때문에 이 모델을 이용하여 테스트 세트와 훈련세트의 성능을 확인
dt = gs.best_estimator_

print(dt.score(test_input, test_target))
#테스트 세트의 점수는 검증세트의 점수보다 낮은 ㅡ것이 일반적임

0.86
